In [1]:
import numpy as np
import math

In [2]:
data = np.loadtxt("data.txt", dtype = "float", delimiter = " ")
X = data.T

In [3]:
U, s, V = np.linalg.svd(X)

In [4]:
def thresh(z,delta):
    return np.sign(z)*(np.abs(z) >= delta)*(np.abs(z)-delta)

In [10]:
def ssvd(X,threu = 1, threv = 1, gamu = 2, gamv =2,u0 =U.T[0], v0 = V.T[0],merr = 10**(-4), niter = 100 ):
    n = X.shape[0]
    d = X.shape[1]
    ud = 1
    vd = 1
    iters = 0
    SST = np.sum(X*X)
    while (ud > merr or vd > merr):
        print(iters)
        iters = iters +1
         # Updating v
        z =  X.T @ u0
        winv = np.abs(z)**gamv
        sigsq = np.abs(SST - np.sum(z*z))/(n*d-d)
        cand = z*winv
        delt = np.sort(np.append(np.abs(cand),0))
        delt_uniq = np.unique(delt)
        Bv = np.ones(len(delt_uniq)-1)*float("inf")
        ind = np.where(winv>10^(-8))
        cand1 = cand[ind]
        winv1 = winv[ind]
        for i in range(len(Bv)):
            temp2 = thresh(cand1,delta = delt_uniq[i])
            temp2 = temp2/winv1
            temp3 = np.zeros(d)
            temp3[ind] = temp2
            Bv[i] = np.sum((X - u0.reshape(len(u0),1) @ temp3.reshape(1,len(temp3)))**2)/sigsq + np.sum(temp2!=0)*math.log(n*d)
        Iv = min(np.where(Bv== min(Bv)))  
        th = delt_uniq[Iv]
        temp2 = thresh(cand1,delta = th)
        temp2 = temp2/winv1
        v1 = np.zeros(d)
        v1[ind] = temp2
        v1 = v1/((np.sum(v1*v1))**0.5) #v_new
        
        # Updating u
        z = X @ v1
        winu = np.abs(z)**gamu
        sigsq = np.abs(SST - np.sum(z*z))/(n*d-n)
        cand = z*winu
        delt = np.sort(np.append(np.abs(cand),0))
        delt_uniq = np.unique(delt)
        Bu = np.ones(len(delt_uniq)-1)*float("inf")
        ind = np.where(winu > 10^(-8))
        cand1 = cand[ind]
        winu1 = winu[ind]
        for i in range(len(Bu)):
            temp2 = thresh(cand1,delta = delt_uniq[i])
            temp2 = temp2/winu1
            temp3 = np.zeros(n)
            temp3[ind] = temp2
            Bu[i] = np.sum((X - temp3.reshape(len(temp3),1) @ v1.reshape(1,len(v1)))**2)/sigsq + np.sum(temp2!=0)*math.log(n*d)
        Iu = min(np.where(Bu==min(Bu)))
        th = delt_uniq[Iu]
        temp2 = thresh(cand1,delta = th)
        temp2 = temp2/winu1
        u1 = np.zeros(n)
        u1[ind] =  temp2
        u1 = u1/((np.sum(u1*u1))**0.5)
        
        
        ud = np.sum((u0-u1)*(u0-u1))**0.5
        vd = np.sum((v0-v1)*(v0-v1))**0.5
        
        if iters > niter :
            print("Fail to converge! Increase the niter!")
            break
        
        u0 = u1
        v0 = v1
    return u1,v1,iters

In [11]:
res = ssvd(X)

0
1
2
3
4
5


In [12]:
res 

(array([-0.18018641, -0.2111319 , -0.10737516, -0.15574824, -0.15112323,
        -0.16025416, -0.19018526, -0.16695614, -0.19288482, -0.1101655 ,
        -0.13320652, -0.16567419, -0.18539703, -0.13521102, -0.17296825,
        -0.11586789, -0.19550012, -0.06831533, -0.17865369, -0.19547649,
         0.09094302,  0.04679269,  0.08135216,  0.0319133 ,  0.05270603,
         0.0326062 ,  0.08880033,  0.07646915,  0.07221645,  0.04541018,
         0.06393339,  0.0096247 , -0.01926316,  0.16077355,  0.15437588,
         0.15564448,  0.13808241,  0.14590629,  0.15895225,  0.16920658,
         0.17449384,  0.15142279,  0.1758557 ,  0.13815952,  0.14764779,
         0.14680345,  0.1735995 ,  0.14234025,  0.12499649,  0.1455915 ,
         0.07617165, -0.03848219, -0.04149155,  0.0123531 , -0.        ,
        -0.11083854]),
 array([ 0.       ,  0.0126103,  0.       , ...,  0.       ,  0.       ,  0.       ]),
 6)